<a href="https://colab.research.google.com/github/ipavlopoulos/diagnostic_captioning/blob/master/DC_show_n_tell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Medical Image To Diagnostic Text
---

### Use the IU-Xray dataset, including radiology XRays along with their human-generated findings/impressions, to train the Show & Tell Encoder-Decoder model. 

The code is based on https://machinelearningmastery.com/develop-a-caption-generation-model-in-keras/ while future work will  i) add visual/semantic attention, ii) experiment with first words given, iii) decode hierarchicaly/consecutive.

In [44]:
# Download the dataset and put in proper folder to use it.
!git clone https://github.com/nlpaueb/bio_image_caption.git
!python bio_image_caption/SiVL19/get_iu_xray.py
!cp -n -r iu_xray /usr/local/share/jupyter/nbextensions/google.colab/iu_xray

fatal: destination path 'bio_image_caption' already exists and is not an empty directory.
--2019-08-08 10:29:16--  https://openi.nlm.nih.gov/imgs/collections/NLMCXR_png.tgz
Resolving openi.nlm.nih.gov (openi.nlm.nih.gov)... 130.14.52.157, 2607:f220:41e:7052::157
Connecting to openi.nlm.nih.gov (openi.nlm.nih.gov)|130.14.52.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1360814128 (1.3G) [application/x-gzip]
Saving to: ‘iu_xray/NLMCXR_png.tgz’

NLMCXR_png.tgz      100%[===================>]   1.27G  74.1MB/s    in 18s     

2019-08-08 10:29:34 (70.6 MB/s) - ‘iu_xray/NLMCXR_png.tgz’ saved [1360814128/1360814128]

--2019-08-08 10:29:34--  https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz
Resolving openi.nlm.nih.gov (openi.nlm.nih.gov)... 130.14.52.157, 2607:f220:41e:7052::157
Connecting to openi.nlm.nih.gov (openi.nlm.nih.gov)|130.14.52.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1112632 (1.1M) [application/x-gzi

In [13]:
# imports
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
import os
import json
import numpy as np
import pandas as pd
import re
import time
from glob import glob
from PIL import Image
import pickle
from os import listdir
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import GRU, LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalMaxPooling2D
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# `Parse the texts and any tags, even if they aren't used.` 

In [0]:
# download the image tags
DATA_PATH = "iu_xray/"
DATA_IMAGES_PATH = os.path.join(DATA_PATH, "iu_xray_images")
iuxray_major_tags = json.load(open(DATA_PATH+"iu_xray_major_tags.json"))
iuxray_auto_tags = json.load(open(DATA_PATH+"iu_xray_auto_tags.json"))
iuxray_captions = json.load(open(DATA_PATH+"iu_xray_captions.json"))

In [46]:
# parse the captions
iuxray_ids = list(iuxray_captions.keys())
iuxray_caption_texts = [iuxray_captions[iid].split() for iid in iuxray_ids]
iuxray_captions_num = len(iuxray_ids)
print ("# texts: {0}".format(iuxray_captions_num))
raw_texts = [" ".join(t) for t in iuxray_caption_texts]
print (len(raw_texts), len(set(raw_texts)))

# texts: 7430
7430 3066


# `Map image/captions to the patient they correspond. `

In [47]:
patient_images = {}
for visit in iuxray_ids:
  patient = visit[3:].split("_")[0]
  if patient in patient_images:
    patient_images[patient].append(visit)
  else:
    patient_images[patient] = [visit]

# a model should be reading both images per patient while excluding other patients
iuxray_ids_img1 = [patient_images[patient][0] for patient in patient_images if len(patient_images[patient])==1]
iuxray_ids_img2 = [patient_images[patient][0] for patient in patient_images if len(patient_images[patient])==2]
iuxray_ids_img3 = [patient_images[patient][0] for patient in patient_images if len(patient_images[patient])>2]
print ("#patients with 1, 2, or more images:", len(iuxray_ids_img1), len(iuxray_ids_img2), len(iuxray_ids_img3))
#print (patient_images)

#patients with 1, 2, or more images: 435 3195 196


# `Download a sentence tokenizer and preprocess the text.`

In [0]:
train_path = os.path.join(DATA_PATH, "train_images.tsv")
test_path = os.path.join(DATA_PATH, "test_images.tsv")
train_data = pd.read_csv(train_path, sep="\t", header=None) 
test_data = pd.read_csv(test_path, sep="\t", header=None) 
train_data.columns = ["id", "caption"]
test_data.columns = ["id", "caption"]
start, end, sentence_token = "startsequence", "endsequence", " endofsentence "

def preprocess(text, start=start, end=end, sentence_token=sentence_token):
  if sentence_token is not None:
    sentences = nltk.tokenize.sent_tokenize(text)
    sentences = [s for s in sentences if len(s)>5]
    text = sentence_token.join(sentences)
  text = text.lower()
  return start + ' ' + text + ' ' + end

***Merge image/captions of same patient and use only patients with 2 images:***

In [49]:
# use the patient DB to create the datasets (one caption per patient)
image_captions = dict(zip(train_data.id.to_list()+test_data.id.to_list(), train_data.caption.to_list()+test_data.caption.to_list()))
print ("# captions", len(image_captions))
patient_captions = {patient:[image_captions[img] for img in patient_images[patient]] for patient in patient_images}
print ("# patients", len(patient_captions))
# discard patients without both XRays
ids = list(patient_captions.keys())
for patient in ids:
  if len(patient_captions[patient])!=2:
    del patient_captions[patient], patient_images[patient]
  else:
    patient_captions[patient] = preprocess(patient_captions[patient][0])
    patient_images[patient] = [os.path.join(DATA_IMAGES_PATH, img_name) for img_name in patient_images[patient]] 

ids = list(patient_captions.keys())
random.shuffle(ids)
sample_size = int(len(ids)*.1)
train_ids = ids[:-sample_size]
test_ids = ids[-sample_size:]
print ("# train/test: ", len(train_ids), len(test_ids))

# captions 7430
# patients 3826
# train/test:  2876 319


# `Set up the VGG19 encoder and encode all the images. `

In [50]:
in_layer = Input(shape=(224, 224, 3))
encoder = VGG16(include_top=False, input_tensor=in_layer)
print(encoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
def encode(img_path):
  image = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
  # convert the image pixels to a numpy array
  image = keras.preprocessing.image.img_to_array(image)
  # prepare the image for the model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = keras.applications.densenet.preprocess_input(image)
  return encoder.predict(image, verbose=0)

In [52]:
from tqdm import tqdm
train_encoded_images = {}
for pid in tqdm(train_ids):
  train_encoded_images[pid] = [encode(img_path) for img_path in patient_images[pid]]

100%|██████████| 2876/2876 [01:14<00:00, 38.64it/s]


In [53]:
test_encoded_images = {}
for pid in tqdm(test_ids):
  test_encoded_images[pid] = [encode(img_path) for img_path in patient_images[pid]]

test_captions = {pid:patient_captions[pid] for pid in test_ids}

100%|██████████| 319/319 [00:08<00:00, 38.37it/s]


In [0]:
pickle.dump(train_encoded_images, open("train_encoded_images.pkl", "wb"))
pickle.dump(test_encoded_images, open("test_encoded_images.pkl", "wb"))
pickle.dump(patient_images, open("patient_images.pkl", "wb"))

In [0]:
train_captions = {pid:patient_captions[pid] for pid in train_ids}
pickle.dump(patient_captions, open("patient_captions.pkl", "wb"))

# `Compute the Vocabulary and make a data processing method.`

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts((train_captions.values()))
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 1807


In [0]:
def create_sequences(tokenizer, caption, image1, image2, max_length):
	Ximages1,Ximages2, XSeq, y = list(), list(), list(),list()
	vocab_size = len(tokenizer.word_index) + 1
	# integer encode the description
	seq = tokenizer.texts_to_sequences([caption])[0]
	# split one sequence into multiple X,y pairs
	for i in range(1, len(seq)):
		# select
		in_seq, out_seq = seq[:i], seq[i]
		# pad input sequence
		in_seq = keras.preprocessing.sequence.pad_sequences([in_seq], maxlen=max_length)[0]
		# encode output sequence
		out_seq = keras.utils.to_categorical([out_seq], num_classes=vocab_size)[0]
		# store
		Ximages1.append(image1)
		Ximages2.append(image2)
		XSeq.append(in_seq)
		y.append(out_seq)
	return [Ximages1, Ximages2, XSeq, y]

#print (np.array(create_sequences(tokenizer, list(train_captions.values())[0], train_encoded_images[train_ids[0]][0][0], train_encoded_images[train_ids[0]][1][0], 58)).shape)

# `Core methods for setting up the model.`

In [0]:
# define the captioning model
def define_model(vocab_size, max_length, loss="categorical_crossentropy"):
  # feature extractor (encoder)
  inputs1 = Input(shape=(7, 7, 512))
  inputs2 = Input(shape=(7, 7, 512))
  fe1 = GlobalMaxPooling2D()(inputs1)
  fe2 = GlobalMaxPooling2D()(inputs2)
  fe3 = Dense(128, activation='relu')(fe1)
  fe4 = Dense(128, activation='relu')(fe2)
  fe5 = concatenate([fe3,fe4])
  fe6 = RepeatVector(max_length)(fe5)
  # embedding
  inputs3 = Input(shape=(max_length,))
  emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs3)
  emb3 = GRU(128, return_sequences=True)(emb2)
  emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
  # merge inputs
  merged = concatenate([fe6, emb4])  
  # language model (decoder)
  lm2 = GRU(256)(merged)
  lm3 = Dense(128, activation='relu')(lm2)
  outputs = Dense(vocab_size, activation='softmax')(lm3)
  # tie it together [image, seq] [word]
  model = Model(inputs=[inputs1,inputs2, inputs3], outputs=outputs)
  model.compile(loss=loss, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
  # loss could also be e.g. nltk.translate.bleu_score.sentence_bleu
  print(model.summary())
  plot_model(model, show_shapes=True, to_file='plot.png')
  return model

In [0]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photos, max_length):
	# seed the generation process
	in_text = start
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photos[0],photos[1],sequence], verbose=0)
		# convert probability to integer
		yhat = np.argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == end:
			break
	return in_text

# generate a description for an image
def generate_desc_beam(model, tokenizer, photos, max_length, beam_size=10):
  in_text = [start]
  start_word = [[start, 0.0]]
  while len(start_word[0][0]) < max_length:
    tmp = []
    for s in start_word:
      sequence = tokenizer.texts_to_sequences(s[0])
      sequence = keras.preprocessing.sequence.pad_sequences([sequence[0]], maxlen=max_length)
      yhat = model.predict([photos[0],photos[1], np.array(sequence)], verbose=0)
      word_yhat = np.argsort(yhat[0])[-beam_size:]
      for w in word_yhat:
        nextcap, prob = s[0], s[1]
        nextcap+= ' ' + word_for_id(w, tokenizer)
        prob += yhat[0][w]
        #print (nextcap, prob)
        tmp.append([nextcap, prob])    
    start_word = tmp
    start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
    start_word = start_word[-beam_size:]    
  start_word = start_word[-1][0]
  intermediate_caption = start_word
  final_caption = []    
  for i in intermediate_caption.split():
    if i != end: final_caption.append(i)
    else: break
  final_caption = ' '.join(final_caption[1:])
  return final_caption   
#bleu = evaluate_n_visualise(loaded_model, test_captions, test_encoded_images, tokenizer, max_length, beam=10)

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		actual.append([desc.split()])
		predicted.append(yhat.split())
	# calculate BLEU score
	bleu = corpus_bleu(actual, predicted)
	return bleu

In [0]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(captions, image_tuples, tokenizer, max_length, n_step, validation=False, validation_num=None):
  while 1:
    # iterate over patients - hold some for validation
    patients = list(captions.keys())
    if validation:
      assert validation_num>0
      patients = patients[-validation_num:]
    elif not validation:
      if validation_num>0:
        patients = patients[:-validation_num]
        
    for i in range(0, len(patients), n_step):
      Ximages1, Ximages2, XSeq, y = list(), list(), list(),list()
      for j in range(i, min(len(patients), i+n_step)):
        patient_id = patients[j]
        # retrieve text input
        caption = captions[patient_id]
        # generate input-output pairs (many images in each batch)
        img1 = image_tuples[patient_id][0][0]
        img2 = image_tuples[patient_id][1][0]
        in_img1, in_img2, in_seq, out_word = create_sequences(tokenizer, caption, img1, img2, max_length)
        for k in range(len(in_img1)):
          Ximages1.append(in_img1[k])
          Ximages2.append(in_img2[k])
          XSeq.append(in_seq[k])
          y.append(out_word[k])
        # yield this batch of samples to the model
        #print (array(Ximages1).shape)
      yield [np.array(Ximages1), np.array(Ximages2), np.array(XSeq)], np.array(y)

In [0]:
# evaluate the skill of the model
def evaluate_n_visualise(model, descriptions, photos, tokenizer, max_length, size=5, beam=0):
  actual, predicted = list(), list()
  # step over the whole set
  for key, desc in descriptions.items():
    if random.random() > 0.8 :
      # generate description
      if beam>0:
        yhat = generate_desc_beam(model, tokenizer, photos[key], max_length, beam_size=beam)
      else:
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
      # store actual and predicted
      print('Actual:    %s' % desc)
      print('Predicted: %s\n' % yhat)
      actual.append([desc.split()])
      predicted.append(yhat.split())
      if len(actual) >= size: break
  # calculate BLEU score
  bleu = corpus_bleu(actual, predicted)
  return bleu


# `Run the model.`

In [62]:
# define experiment
verbose = 1
n_epochs = 300
max_length = 60
n_patients_per_update = 16
val_len = int(.01 * len(train_ids))
train_len = len(train_ids) - val_len
train_steps = int(train_len / n_patients_per_update)
val_steps = int(val_len / n_patients_per_update)
model_name = 'show_n_tell.e'+str(n_epochs)+'.ml'+str(max_length) + '.ppu'+str(n_patients_per_update) + '.val'+str(val_steps)
print (train_steps, val_steps, model_name)

178 1 show_n_tell.e300.ml60.ppu16.val1


In [0]:
show_n_tell = define_model(vocab_size, max_length, loss="categorical_crossentropy")

# Train
train_gen = data_generator(train_captions, train_encoded_images, tokenizer, max_length, n_patients_per_update, validation=False, validation_num=val_len)
val_gen = data_generator(train_captions, train_encoded_images, tokenizer, max_length, n_patients_per_update, validation=True, validation_num=val_len)
early = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.001, patience=10, verbose=1, mode='auto', restore_best_weights=True)
show_n_tell.fit_generator(train_gen, validation_data=val_gen, steps_per_epoch=train_steps, validation_steps=val_steps, epochs=n_epochs, verbose=verbose, callbacks=[early])

# Save & download
show_n_tell.save(model_name+'.h5')
#from google.colab import files
#files.download(model_name+'.h5')

# `See an example text generation:`

In [88]:
# Load & evaluate
#loaded_model = keras.models.load_model(model_name+'.h5')
loaded_model.summary()
plot_model(loaded_model, show_shapes=True, to_file='model.png')

! mv model.png /usr/local/share/jupyter/nbextensions/google.colab/model.png

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
global_max_pooling2d_15 (Global (None, 512)          0           input_24[0][0]                   
__________________________________________________________________________________________________
global_max_pooling2d_16 (Global (None, 512)          0           input_25[0][0]                   
__________________________________________________________________________________________________
input_26 (

In [90]:
%%html
<img src='/nbextensions/google.colab/model.png' width=1024/>

In [77]:
b4 = evaluate_n_visualise(loaded_model, test_captions, test_encoded_images, tokenizer, max_length, beam=0)
print ("BLEU (@sample):", b4)

Actual:    startsequence no acute disease. endofsentence the heart is normal in size. endofsentence the mediastinum is unremarkable. endofsentence the lungs are clear. endsequence
Predicted: startsequence no acute cardiopulmonary abnormality endofsentence the lungs are clear bilaterally endofsentence specifically no evidence of focal consolidation pneumothorax or pleural effusion cardio mediastinal silhouette is unremarkable endofsentence visualized osseous structures of the thorax are without acute abnormality endsequence

Actual:    startsequence no acute cardiopulmonary abnormalities. endofsentence normal heart size is prominent left ventricular contour. endofsentence unfolding of the thoracic aorta. endofsentence no focal airspace consolidation. endofsentence no pleural effusion or pneumothorax. endofsentence visualized osseous structures are unremarkable appearance. endsequence
Predicted: startsequence no acute cardiopulmonary abnormality endofsentence the lungs are clear bilatera

# `See the diversity & evaluate.`

In [0]:
predicted = list()
actual = list()
for key in test_encoded_images:
    caption = generate_desc(loaded_model, tokenizer, test_encoded_images[key], max_length)
    gold = test_captions [key]
    predicted.append(caption.split())
    actual.append([gold.split()])

In [84]:
denom = len(predicted)
nom_pred = len(set(map(lambda x: " ".join(x), predicted)))
nom_actual = len(set(map(lambda x: " ".join(x[0]), actual)))
print ("diversity@predicted:", nom_pred/denom)
print ("diversity@actual:", nom_actual/denom)

diversity@predicted: 0.13166144200626959
diversity@actual: 0.9341692789968652


In [85]:
# call METEOR, ROUGE, CIDEr, SPICE, 
! git clone https://github.com/salaniz/pycocoevalcap.git || true
from pycocoevalcap.rouge import rouge
from pycocoevalcap.meteor import meteor
from pycocoevalcap.bleu import bleu
from pycocoevalcap.cider import cider

measures = {"ROU":[], "MET":[], "BLU1":[], "BLU2":[], "BLU3":[], "BLU4":[], "SPI":[], "CID":[]}
for pred,act in zip(predicted, actual):
  pred, act = " ".join(pred), " ".join(act[0])
  rou = rouge.Rouge().calc_score([pred], [act])
  blu1, _ = bleu.BleuScorer(n=1, test=pred, refs=[act]).compute_score(option='closest', verbose=0)
  blu2, _ = bleu.BleuScorer(n=2, test=pred, refs=[act]).compute_score(option='closest', verbose=0)
  blu3, _ = bleu.BleuScorer(n=3, test=pred, refs=[act]).compute_score(option='closest', verbose=0)
  blu4, _ = bleu.BleuScorer(n=4, test=pred, refs=[act]).compute_score(option='closest', verbose=0)
  # blu4, _ = bleu.Bleu(4).compute_score({1:[pred]}, {1:[act]})
  cid, _ = cider.CiderScorer(test=pred, refs=[act]).compute_score(option='closest', verbose=0)
  #met, _ = meteor.Meteor().compute_score({1:[pred]}, {1:[act]}) # too slow
  measures["ROU"].append(rou)
  measures["BLU1"].append(blu1)
  measures["BLU2"].append(blu2)
  measures["BLU3"].append(blu3)
  measures["BLU4"].append(blu4)
  measures["CID"].append(cid)

print ("ROUGE:", np.mean(measures["ROU"]))
print ("METEOR:", np.mean(measures["MET"]))
print ("BLEU1:", np.mean(measures["BLU1"]))
print ("BLEU2:", np.mean(measures["BLU2"]))
print ("BLEU3:", np.mean(measures["BLU3"]))
print ("BLEU4:", np.mean(measures["BLU4"]))
print ("CIDEr:", np.mean(measures["CID"]))

Cloning into 'pycocoevalcap'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 751 (delta 2), reused 6 (delta 2), pack-reused 744
Receiving objects: 100% (751/751), 106.72 MiB | 41.61 MiB/s, done.
Resolving deltas: 100% (404/404), done.
ROUGE: 0.3117538312591588
METEOR: nan
BLEU1: 0.32565657693775246
BLEU2: 0.2501687377541738
BLEU3: 0.20029975998237662
BLEU4: 0.16473906865130708
CIDEr: 0.0


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# `Now see what happens per epoch:`

In [0]:
# See what the model produces in each epoch:
show_n_tell = define_model(vocab_size, max_length, loss="categorical_crossentropy")
for i in range(n_epochs):  
  print ("### Running epoch ", i, " ###")
  train_gen = data_generator(train_captions, train_encoded_images, tokenizer, max_length, n_patients_per_update, validation=False, validation_num=val_len)
  val_gen = data_generator(train_captions, train_encoded_images, tokenizer, max_length, n_patients_per_update, validation=True, validation_num=val_len)
  show_n_tell.fit_generator(train_gen, validation_data=val_gen, steps_per_epoch=train_steps, validation_steps=val_steps, epochs=1, verbose=verbose)
  for _ in range(3):
    test_id = random.choice(test_ids)
    generation = generate_desc(show_n_tell, tokenizer, test_encoded_images[test_id], max_length)
    print ("Predicted:", generation)
    print ("Actual   :", test_captions[test_id])
    print ()

In [0]:
# run experiment with repetitions
n_repeats = 1
train_results, test_results = list(), list()
for i in range(n_repeats):
	model = define_model(vocab_size, max_length)
	model.fit_generator(data_generator(train_captions, train_encoded_images, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
	train_score = evaluate_model(model, train_captions, train_encoded_images, tokenizer, max_length)
	train_results.append(train_score)
	print('>%d: train=%f' % ((i+1), train_score))
df = DataFrame()
df['train'] = train_results
print(df.describe())
df.to_csv(model_name+'.csv', index=False)